In [1]:
import logging
import re
import os

import numpy as np
import tifffile
from ome_types import from_tiff, from_xml, to_xml, model, to_dict
from ome_types.model.simple_types import UnitsLength

from mip.utils import listfiles

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from mip.ome import generate_ome_from_qptiff

In [5]:
qptiff_fp = '/diskmnt/Projects/Users/estorrs/multiplex_data/codex/htan/brca/03172023_BRCA_HT397B1-U12/level_2/HT397B1-U12_Scan1.qptiff'
output_fp = '/diskmnt/Projects/Users/estorrs/sandbox/test_pyramidal.ome.tiff'
bbox = (5000, 6000, 5000, 6000)

In [6]:
generate_ome_from_qptiff(qptiff_fp, output_fp, bbox=bbox)

In [7]:
??generate_ome_from_qptiff

In [2]:
qptiff_fp = '/diskmnt/Projects/Users/estorrs/multiplex_data/codex/htan/brca/03172023_BRCA_HT397B1-U12/level_2/HT397B1-U12_Scan1.qptiff'
# qptiff_fp = '/diskmnt/Projects/Users/estorrs/multiplex_data/codex/htan/brca/040623_BRCA_HT397B1-U31/level_2/HT397B1-S1H1A3U31_Scan2.qptiff'

output_fp = '/diskmnt/Projects/Users/estorrs/sandbox/test_pyramidal.ome.tiff'
PHENOCYCLER_PIXELS_PER_MICRON = 1.9604911906033102


In [3]:
pixel_type = 'uint8'
tf = tifffile.TiffFile(qptiff_fp)
s = tf.series[0] # full res tiffs are in the first series
n_channels = s.get_shape()[0]
shape = s.pages[0].shape
subresolutions = 5

bbox = (5000, 6000, 5000, 6000)
# bbox = None
shape

(18720, 19200)

In [4]:
if bbox is None:
    data = np.zeros((shape[1], shape[0], n_channels, 1, 1),
                    dtype=np.uint8 if pixel_type=='uint8' else np.uint16)
else:
    data = np.zeros((bbox[3] - bbox[2], bbox[1] - bbox[0], n_channels, 1, 1),
                    dtype=np.uint8 if pixel_type=='uint8' else np.uint16)
biomarkers = []
for i, p in enumerate(s.pages):
    img = p.asarray()
    d = tifffile.xml2dict(p.description)['PerkinElmer-QPI-ImageDescription']
    biomarker = d['Biomarker']
    biomarkers.append(biomarker)
    if bbox is not None:
        r1, r2, c1, c2 = bbox
        y = r2 - r1
        x = c2 - c1
        img = img[r1:r2, c1:c2]
    img = img.astype(np.float32)
    img -= img.min()
    img /= img.max()
    if pixel_type == 'uint8':
        img *= 255
        img = img.astype(np.uint8)
    else:
        img *= 65535
        img = img.astype(np.uint16) 

    data[..., i, 0, 0] = np.swapaxes(img, 0, 1)
data.shape

(1000, 1000, 25, 1, 1)

In [5]:
o = model.OME()
o.images.append(
    model.Image(
        id='Image:0',
        pixels=model.Pixels(
            dimension_order='XYCZT',
            size_c=n_channels,
            size_t=1,
            size_x=data.shape[0],
            size_y=data.shape[1],
            size_z=data.shape[3],
            type=pixel_type,
            big_endian=False,
            channels=[model.Channel(id=f'Channel:{i}', name=c) for i, c in enumerate(biomarkers)],
            physical_size_x=1 / PHENOCYCLER_PIXELS_PER_MICRON,
            physical_size_y=1 / PHENOCYCLER_PIXELS_PER_MICRON,
            physical_size_x_unit='µm',
            physical_size_y_unit='µm'
        )
    )
)

im = o.images[0]
for i in range(len(im.pixels.channels)):
    im.pixels.planes.append(model.Plane(the_c=i, the_t=0, the_z=0))
im.pixels.tiff_data_blocks.append(model.TiffData(plane_count=len(im.pixels.channels)))
# xml_str = to_xml(o)
metadata = to_dict(to_xml(o))

/tmp/ipykernel_107137/2295163470.py:28: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  metadata = to_dict(to_xml(o))


In [8]:
from einops import rearrange
from skimage.transform import rescale

In [164]:
p = tf.pages[0]
p.compression

<COMPRESSION.LZW: 5>

In [ ]:
# !pip install imagecodecs==2023.3.16

In [9]:
with tifffile.TiffWriter(output_fp, ome=True, bigtiff=True) as out_tif:
    opts = {
        'compression': 'LZW',
    }
    out_tif.write(
        rearrange(data, 'x y c z t -> t c y x z'),
        subifds=subresolutions,
        **opts
    )
    for level in range(subresolutions):
        mag = 2**(level + 1)
        sampled = rescale(data[..., 0, 0], 1 / mag, preserve_range=True, channel_axis=-1)
        sampled = sampled.astype(np.uint8)
        sampled = np.expand_dims(sampled, (-2, -1))
        out_tif.write(
            rearrange(sampled, 'x y c z t -> t c y x z'),
            subfiletype=1,
            **opts
        )
    xml_str = to_xml(o)
    out_tif.overwrite_description(xml_str.encode())

In [129]:
to_dict(to_xml(o))

{'xmlns': 'http://www.openmicroscopy.org/Schemas/OME/2016-06',
 'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 'xsi:schemaLocation': 'http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd',
 'images': [{'id': 'Image:0',
   'pixels': {'big_endian': False,
    'dimension_order': 'XYCZT',
    'id': 'Pixels:8',
    'physical_size_x': 0.5100762527233116,
    'physical_size_y': 0.5100762527233116,
    'size_c': 25,
    'size_t': 1,
    'size_x': 1000,
    'size_y': 1000,
    'size_z': 1,
    'type': 'uint8',
    'physical_size_x_unit': 'µm',
    'physical_size_y_unit': 'µm',
    'physical_size_z_unit': 'µm',
    'time_increment_unit': 's',
    'channels': [{'color': -1,
      'id': 'Channel:0',
      'name': 'DAPI',
      'pinhole_size_unit': 'µm',
      'excitation_wavelength_unit': 'nm',
      'emission_wavelength_unit': 'nm'},
     {'color': -1,
      'id': 'Channel:1',
      'name': 'CD8',
      'pinhole_size_unit': 'µm',


In [32]:
f = open(qptiff_fp, 'rb')
import struct


In [43]:
f.seek(0)

0

In [44]:
header = f.read(4)
byteorder = {b'II': '<', b'MM': '>', b'EP': '<'}[header[:2]]
header, byteorder

(b'II+\x00', '<')

In [45]:
version = struct.unpack(byteorder + 'H', header[2:4])[0]
version

43

In [46]:
offsetsize, zero = struct.unpack(byteorder + 'HH', f.read(4))
offsetsize, zero

(8, 0)

In [56]:
from tifffile import TIFF, TiffPages, TiffPage

In [48]:
TIFF.BIG_LE

<tifffile.TiffFormat 64-bit little-endian>

In [ ]:
pages = TiffPages(self)

In [57]:
??TiffPage

In [53]:
f.seek(offsetsize)

8

In [58]:
from mip.ome import generate_ome_from_qptiff

In [60]:
generate_ome_from_qptiff(qptiff_fp, output_fp, bbox=bbox)

In [54]:
??TiffPage

Object `TiffPage` not found.


In [20]:
tif = TiffFile(qptiff_fp)
# ome = from_xml(tif.ome_metadata)
# im = ome.images[0]

In [23]:
ome = from_tiff(qptiff_fp, parser='lxml')

XMLSyntaxError: Document labelled UTF-16 but has UTF-8 content, line 1, column 38 (<string>, line 1)

In [18]:
from tifffile import TiffFile
from ome_types import from_tiff, from_xml

In [ ]:
tif = TiffFile(output_fp)
ome = from_xml(tif.ome_metadata)
im = ome.images[0]

In [ ]:
im

In [ ]:
im.pixels.channels

In [10]:
from mip.utils import extract_ome_tiff

In [11]:
d = extract_ome_tiff(output_fp)

/diskmnt/Projects/Users/estorrs/miniconda3/envs/multiplex-imaging-pipeline/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [12]:
d.keys()

dict_keys(['DAPI', 'CD8', 'Her2 (D)', 'GATA3 (D)', 'cKIT-(D)', 'Pan-Cytokeratin', 'GLUT1-(D)', 'Podoplanin', 'CD68 (D)', 'HLA-DR', 'Keratin 14', 'FoxP3', 'MGP-(D)', 'CD20-(D)', 'SMA-(D)', 'Ki67', 'Vimentin-(D)', 'PR-(D)', 'Bap1 (D)', 'CD45 (D)', 'ER', 'CD31', 'COX6c (D)', 'CK19', 'PLAT/tPA (D)'])